In [50]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

In [49]:
# Loading BERT model all-mpnet-base-v2
model = SentenceTransformer('all-mpnet-base-v2')

In [51]:
# OpenSearch instance parameters
host = 'localhost'
port = 9200
auth = ('admin', 'admin')

# Create the client with SSL/TLS enabled and disable warnings
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],    
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_show_warn = False,
)

In [62]:
# Testing connection to OpenSearch
client.info()

{'name': 'opensearch',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ghgK1h_FSeefVD9yf6_gag',
 'version': {'distribution': 'opensearch',
  'number': '2.11.1',
  'build_type': 'tar',
  'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752',
  'build_date': '2023-11-29T21:43:10.135035992Z',
  'build_snapshot': False,
  'lucene_version': '9.7.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [63]:
# Query embedding
query = "where does hypotonia typically appear?"
query_emb = model.encode(query)

In [64]:
# Defining the knn query parameters
search_query = {    
    "query": {
        "knn": {
            "embedding": {
                "vector": query_emb.tolist(),
                "k": 3
            }
        }
    }
}

In [68]:
# Send a knn query to OpenSearch
results = client.search(index="pubmed_index", body=search_query)

In [69]:
# Helper function to view the reposes easily
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        pmid = hit['_source']['pmid']
        chunk_id = hit['_source']['chunk_id']  
        chunk = hit['_source']['chunk']      
        pretty_output = (f"\nID: {id}\nPMID: {pmid}\nChunk ID: {chunk_id}\nText: {chunk}")
        print(pretty_output)

In [70]:
pretty_response(results)


ID: nwDXHI0BUtdyfxKMHCHL
PMID: 23414618
Chunk ID: 1
Text: LPFC. In addition, it is suggested that low hemodynamic activities in this area are correlated with ADHD.

ID: owDXHI0BUtdyfxKMdjde
PMID: 26328655
Chunk ID: 0
Text: Idiopathic hypoparathyroidism (IHP) is accompanied by cognitive impairment. We report the case of a 70-year-old IHP patient with cognitive disturbance. Brain computed tomography showed bilateral calcification in basal ganglia, thalamus, and cerebellum. Neuropsychological assessment revealed low scores for intelligence, memory, and perseverative errors. Brain positron emission tomography showed a significant reduction in [(18)F]-Fludeoxyglucose (FDG) uptake in bilateral frontal, left temporal and parietal cortices, along with a marked reduction in [(11)C]-flumazenil binding in left frontal, temporal, parietal, and bilateral cerebellum. These findings suggest cognitive impairment in IHP may be ascribed to GABAergic dysfunction, thus leading to, or coexisting with, cer